In [ ]:
import os
import shutil
from tqdm import tqdm

In [ ]:
from avapi.carla import CarlaScenesManager


cpath = os.path.join("/data/shared/CARLA/multi-agent-intersection/")
# cpath = os.path.join("/data/shared/CARLA/multi-agent-trust/")
CSM = CarlaScenesManager(cpath)
idx = 0
CDM = CSM.get_scene_dataset_by_index(idx)
vid_folder = f"videos_intersection_{idx}"

print(CSM.scenes)
print(f"{len(CDM)} frames")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from avapi.visualize.snapshot import show_image_with_boxes, show_lidar_bev_with_boxes


agent = 0
frame_idx = 20

# load image
camera_sensor = "camera-0"
frame = CDM.get_frames(sensor=camera_sensor, agent=agent)[frame_idx]
img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
show_image_with_boxes(img=img, boxes=[], inline=True)

# load point cloud
lidar_sensor = "lidar-0"
frame = CDM.get_frames(sensor=lidar_sensor, agent=agent)[frame_idx]
pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor, agent=agent)
objs = CDM.get_objects(frame=frame, sensor=camera_sensor, agent=agent)
show_lidar_bev_with_boxes(pc=pc, boxes=objs)

# run concave hull algorithm
hull = pc.concave_hull_bev(concavity=1, length_threshold=10)
fig, ax = plt.subplots()
ax.plot(-1 * hull.boundary[:, 1], hull.boundary[:, 0], linewidth=3)
polygon = Polygon(
    np.array([-1, 1]) * hull.boundary[:, [1, 0]],
    closed=True,
    facecolor="blue",
    alpha=0.25,
)
ax.add_patch(polygon)
plt.axis("scaled")
plt.axis("off")
plt.show()

## Test Multi-Agent Tracking and Fusion

In [ ]:
from datetime import timedelta

from stonesoup.types.groundtruth import GroundTruthState


def object_to_stone_soup_truth(obj, t_start):
    ts = t_start + timedelta(seconds=obj.t)
    xx, xy, xz = obj.position.x
    h, w, l = obj.box.size
    vx, vy, vz = obj.velocity.x
    er, ep, ey = obj.attitude.euler
    state = [xx, vx, xy, vy, xz, vz, h, w, l, er, ep, ey]
    metadata = {
        "object_type": obj.obj_type,
        "object_ID": obj.ID,
        "occlusion": obj.occlusion,
    }
    return GroundTruthState(state, timestamp=ts, metadata=metadata)

In [ ]:
from datetime import datetime, timedelta

from stonesoup.types.groundtruth import GroundTruthPath

from avstack.datastructs import DataContainer
from avstack.geometry import GlobalOrigin3D
from avstack.modules.clustering import BasicClusterer
from avstack.modules.perception.object3d import MMDetObjectDetector3D
from avstack.modules.perception.detections import BoxDetection
from avstack.modules.tracking.tracker3d import BasicBoxTracker3D
from avstack.modules.tracking.stonesoup import StoneSoupKalmanTracker3DBox
from avstack.modules.tracking.multisensor import MeasurementBasedMultiTracker

from mate import plotting
from mate.estimator import TrustEstimator
from mate.utils import BetaDistWriter, PsmWriter


# init models
t_start = datetime.now()
agents = list(range(4))
agent_is_static = {
    i: "static" in CDM.get_agent(frame=1, agent=i).obj_type for i in agents
}
n_static = sum(list(agent_is_static.values()))
print(
    "There are {} agents\n   {} mobile, {} static".format(
        len(agents), len(agents) - n_static, n_static
    )
)
clusterer = BasicClusterer(assign_radius=0.5)
percep_veh = MMDetObjectDetector3D(
    model="pointpillars", dataset="carla-vehicle", gpu=0, thresh_duplicate=1.0
)
percep_inf = MMDetObjectDetector3D(
    model="pointpillars", dataset="carla-infrastructure", gpu=0, thresh_duplicate=1.0
)
percep_col = None
trackers = {agent: StoneSoupKalmanTracker3DBox(t0=t_start) for agent in agents}
# trackers["central"] = MeasurementBasedMultiTracker(
#     tracker=BasicBoxTracker3D(name="trackerformulti", run_clustering=False)
# )
trackers["central"] = MeasurementBasedMultiTracker(
    tracker=StoneSoupKalmanTracker3DBox(t0=t_start, name="trackerformulti")
)
trackers["collab"] = BasicBoxTracker3D()
timestamps = [t_start]
truths = {"global_visible": {}, "global": {}, "agent": {agent: {} for agent in agents}}
visible_times = {"first": {}, "last": {}}

# init data structures
ss_tracks = {}
dets = {agent: [] for agent in agents}
tracks = {"central": [], **{agent: [] for agent in agents}}
timestamps_all = {agent: [] for agent in agents}
imgs_all = {agent: [] for agent in agents}
pcs_all = {agent: [] for agent in agents}
dets_all = {agent: [] for agent in agents}
dets_all["collab"] = []
tracks_all = {agent: [] for agent in agents}
tracks_all["central"] = []
tracks_all["collab"] = []
agent_0_frames = CDM.get_frames(sensor="lidar-0", agent=0)[1:-1]
platforms_all = {agent: [] for agent in agents}

# flags for this run
run_perception_model = False
run_distributed_perception = True
run_distributed_tracking = True
run_centralized_tracking = True
run_collaborative_perception = False
run_collaborative_tracking = False
run_trust_estimation = False
debug_trust = True
psm_agent_file = "psm_agent_log.txt"
open(psm_agent_file, "w").close()
psm_track_file = "psm_track_log.txt"
open(psm_track_file, "w").close()
agent_trust_file = "trust_agent_log.txt"
open(agent_trust_file, "w").close()
track_trust_file = "trust_track_log.txt"
open(track_trust_file, "w").close()
if os.path.exists("debug_figures"):
    shutil.rmtree("debug_figures")

if run_trust_estimation:
    trustest = TrustEstimator()
    trust_burnin = 5

# run loop
n_frames_max = 100
ego_agent = agents[0]
for i_frame, frame in enumerate(
    tqdm(agent_0_frames[: min(n_frames_max, len(agent_0_frames))])
):
    fovs = {}
    fovs_local = {}
    platforms = {}
    perception_input = {}

    ###############################################
    # RUN LOOP OVER AGES
    ###############################################
    objs_in_view = set()
    for agent in agents:
        ###############################################
        # GET DATA
        ###############################################
        lidar_sensor = "lidar-0"
        camera_sensor = "camera-0"
        timestamp = CDM.get_timestamp(frame=frame, sensor=lidar_sensor, agent=agent)
        img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
        pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor, agent=agent)
        imgs_all[agent].append(img)
        pcs_all[agent].append(pc)
        objs = CDM.get_objects(frame=frame, sensor=lidar_sensor, agent=agent)
        calib = CDM.get_calibration(frame=frame, sensor=lidar_sensor, agent=agent)
        fovs_local[agent] = pc.concave_hull_bev(
            concavity=1, length_threshold=4, in_global=False
        )
        fovs[agent] = pc.concave_hull_bev(
            concavity=1, length_threshold=4, in_global=True
        )
        platforms[agent] = calib.reference
        platforms_all[agent].append(calib.reference)

        ###############################################
        # SAVE AGENT GROUND TRUTH IN SS FORMAT
        ###############################################
        for obj in objs:
            objs_in_view.add(obj.ID)
            if obj.ID not in truths["agent"][agent]:
                truths["agent"][agent][obj.ID] = GroundTruthPath()
            truths["agent"][agent][obj.ID].append(
                object_to_stone_soup_truth(obj, t_start)
            )

        ###############################################
        # DISTRIBUTED PERCEPTION
        ###############################################
        if run_distributed_perception:
            if run_perception_model:
                if agent_is_static[agent]:
                    dets[agent] = percep_inf(pc)
                else:
                    dets[agent] = percep_veh(pc)
            else:  # use groundtruth
                dets[agent] = DataContainer(
                    frame=frame,
                    timestamp=timestamp,
                    source_identifier="perception",
                    data=[
                        BoxDetection(
                            source_identifier="perception",
                            box=obj.box3d,
                            reference=obj.reference,
                            obj_type=obj.obj_type,
                            score=1.0,
                        )
                        for obj in objs
                        # if obj.ID == 15
                    ],
                )
            dets_all[agent].append(dets[agent])

        ###############################################
        # DISTRIBUTED TRACKING USING DISTRIBUTED PERCEP
        ###############################################
        if run_distributed_tracking:
            assert run_distributed_perception
            tracks_this = trackers[agent](
                dets[agent], platform=calib.reference, calibration=calib
            )
            tracks[agent] = tracks_this.apply_and_return("getattr", "box3d")
            tracks_all[agent].append([box3d.copy() for box3d in tracks[agent]])

    ###############################################
    # COLLABORATIVE PERCEPTION/TRACKING
    ###############################################
    if run_collaborative_perception:
        raise NotImplementedError

    ###############################################
    # CENTRALIZED TRACKING USING DISTRIBUTED PERCEP
    ###############################################
    # run central tracker on all detections
    if run_centralized_tracking:
        # Run trust model
        agent_positions = {
            ID: platforms[ID].integrate(start_at=GlobalOrigin3D).x
            for ID, p in platforms.items()
        }
        obj_positions = [
            obj.position.change_reference(GlobalOrigin3D, inplace=False).x
            for obj in objs
        ]
        dets_global = {
            ID: dets_agent.apply_and_return(
                "change_reference", GlobalOrigin3D, inplace=False
            )
            for ID, dets_agent in dets.items()
        }
        tracks_global = {}
        for ID, tracks_agent in tracks.items():
            # predict central tracks
            tracks_this = []
            if ID == "central":
                for track in tracks_agent:
                    try:
                        # for AVstack tracks
                        track.predict(timestamp)
                        tracks_this.append(track)
                    except AttributeError:
                        # for StoneSoup tracks
                        dt_ts = trackers["central"].t0 + timedelta(seconds=timestamp)
                        track_predicted = trackers["central"]._predictor.predict(
                            track, timestamp=dt_ts
                        )
                        track_predicted.id = track.id
                        trackers["central"]._augment_track(
                            track_predicted, track.box3d.reference
                        )
                        tracks_this.append(track_predicted.box3d)
            else:
                tracks_this = tracks_agent

            # convert to global origin
            tracks_global_this = []
            for track in tracks_this:
                tracks_global_this.append(
                    track.change_reference(GlobalOrigin3D, inplace=False)
                )
            tracks_global[ID] = tracks_global_this
        tracks_global_no_central = {
            ID: tracks for ID, tracks in tracks_global.items() if ID != "central"
        }

        # allow for tracker burnin
        if run_trust_estimation:
            if i_frame >= trust_burnin:
                clusters, psms_agents, psms_tracks = trustest(
                    agent_positions,
                    fovs,
                    dets_global,
                    tracks_global.get("central", []),
                    tracks_global_no_central,
                )

                # debugging
                if debug_trust:
                    PsmWriter.write(frame, psms_agents, psm_agent_file)
                    PsmWriter.write(frame, psms_tracks, psm_track_file)
                    BetaDistWriter.write(frame, trustest.agent_trust, agent_trust_file)
                    BetaDistWriter.write(frame, trustest.track_trust, track_trust_file)
                    plotting.plot_agents_detections(
                        agent_positions,
                        fovs,
                        dets_global,
                        show=False,
                        save=True,
                        fig_dir="debug_figures/detections",
                        suffix=f"-frame-{frame}",
                        extension="png",
                    )
                    plotting.plot_agents_tracks(
                        agent_positions,
                        fovs,
                        tracks["central"],
                        show=False,
                        save=True,
                        fig_dir="debug_figures/tracks",
                        suffix=f"-frame-{frame}",
                        extension="png",
                    )
                    plotting.plot_trust(
                        trustest,
                        show=False,
                        save=True,
                        fig_dir="debug_figures/trust",
                        use_subfolders=True,
                        suffix=f"-frame-{frame}",
                        extension="png",
                    )
                    plt.close()

        # Run tracking
        tracks["central"] = trackers["central"](
            detections=dets_global,
            fovs=fovs,
            platforms=[GlobalOrigin3D] * len(fovs),
        )
        tracks_all["central"].append(
            [track.box3d.copy() for track in tracks["central"]]
        )

    ###############################################
    # SAVE GLOBAL GROUND TRUTH IN SS FORMAT
    ###############################################
    objs_global = CDM.get_objects_global(
        frame=frame, include_agents=True, ignore_static_agents=True
    )
    ts = CDM.get_timestamp(frame=frame, sensor=lidar_sensor, agent=agents[0])
    ss_timestamp = t_start + timedelta(seconds=ts)
    for obj in objs_global:
        obj.t = timestamp  # HACK
        # save all object states
        if obj.ID not in truths["global"]:
            truths["global"][obj.ID] = GroundTruthPath()
        truths["global"][obj.ID].append(object_to_stone_soup_truth(obj, t_start))
        # save objects in view
        if (obj.ID in objs_in_view) or (obj.ID > 10000):
            if obj.ID not in visible_times["first"]:
                visible_times["first"][obj.ID] = ss_timestamp
            if ss_timestamp < visible_times["first"][obj.ID]:
                raise RuntimeError("New timestamp is earlier than 'first'")
            visible_times["last"][obj.ID] = ss_timestamp
    timestamps.append(ss_timestamp)


###############################################
# MASSAGE THE SET OF VISIBLE OBJECTS
###############################################
for obj_ID in truths["global"]:
    if obj_ID in visible_times["first"]:
        truths["global_visible"][obj_ID] = GroundTruthPath()
        for state in truths["global"][obj_ID]:
            if (
                visible_times["first"][obj_ID]
                <= state.timestamp
                <= visible_times["last"][obj_ID]
            ):
                truths["global_visible"][obj_ID].append(state)

In [ ]:
from mate import plotting

# plotting.plot_agents(agent_positions, fovs)
# plotting.plot_agents_objects(agent_positions, fovs, obj_positions, fps=[], fns=[])
if run_trust_estimation:
    plotting.plot_agents_detections(agent_positions, fovs, dets_global)
    plotting.plot_agents_clusters(agent_positions, fovs, clusters)
    plotting.plot_agents_tracks(agent_positions, fovs, tracks["central"])
    plotting.plot_trust(trustest)

In [ ]:
from ordered_set import OrderedSet
from stonesoup.plotter import AnimatedPlotterly

# plot in global
truths_this = OrderedSet(truths["global_visible"].values())
plotter = AnimatedPlotterly(timestamps, tail_length=0.3, sim_duration=4)
plotter.plot_ground_truths(truths_this, [0, 2])
plotter.plot_tracks(
    tracks["central"].data, uncertainty=True, plot_history=True, mapping=[0, 2]
)
plotter.fig

In [ ]:
from stonesoup.dataassociator.tracktotrack import TrackToTruth
from stonesoup.metricgenerator.manager import MultiManager
from stonesoup.plotter import MetricPlotter
from stonesoup.measures import Euclidean
from stonesoup.metricgenerator.metrictables import SIAPTableGenerator
from stonesoup.metricgenerator.ospametric import OSPAMetric
from stonesoup.metricgenerator.tracktotruthmetrics import SIAPMetrics
from stonesoup.metricgenerator.uncertaintymetric import SumofCovarianceNormsMetric


tracking_filters = ["Baseline"]

ospa_generators = [
    OSPAMetric(
        c=40,
        p=1,
        generator_name=f"{tracking_filter} OSPA metrics",
        tracks_key=f"tracks_{tracking_filter}",
        truths_key="truths",
    )
    for tracking_filter in tracking_filters
]

siap_generators = [
    SIAPMetrics(
        position_measure=Euclidean((0, 2)),
        velocity_measure=Euclidean((1, 3)),
        generator_name=f"{tracking_filter} SIAP metrics",
        tracks_key=f"tracks_{tracking_filter}",
        truths_key="truths",
    )
    for tracking_filter in tracking_filters
]

uncertainty_generators = [
    SumofCovarianceNormsMetric(
        generator_name=f"{tracking_filter} OSPA metrics",
        tracks_key=f"tracks_{tracking_filter}",
    )
    for tracking_filter in tracking_filters
]
associator = TrackToTruth(association_threshold=30)

generators = ospa_generators + siap_generators + uncertainty_generators
metric_manager = MultiManager(generators, associator=associator)

metric_manager.add_data(
    {
        "truths": OrderedSet(truths["global_visible"].values()),
        "tracks_Baseline": tracks["central"].data,
    }
)
metrics = metric_manager.generate_metrics()

In [ ]:
fig1 = MetricPlotter()
fig1.plot_metrics(metrics, metric_names=["OSPA distances"])

In [ ]:
siap_metrics = metrics["Baseline SIAP metrics"]
siap_averages_baseline = {
    siap_metrics.get(metric)
    for metric in siap_metrics
    if metric.startswith("SIAP") and not metric.endswith(" at times")
}

_ = SIAPTableGenerator(siap_averages_baseline).compute_metric()
print("\n\nSIAP metrics for Baseline:")

## Visualize

In [ ]:
from utils import make_agent_movies, make_central_movie, make_collab_movie

# agent-specific movie
if run_distributed_tracking:
    print("Making distributed agent movies")
    extent = [[-70, 70], [-70, 70], [-100, 100]]
    for agent in agents:
        # AVstack movie
        make_agent_movies(
            imgs=imgs_all[agent],
            pcs=pcs_all[agent],
            dets=dets_all[agent],
            tracks=tracks_all[agent],
            agent=agent,
            extent=extent,
            percep_movies=False,
            track_movies=True,
            img_movies=True,
            bev_movies=True,
            vid_folder=vid_folder,
        )


# central tracking movie
if run_centralized_tracking:
    print("Making centralized tracking movies")
    extent = [[-150, 20], [-80, 40], [-100, 100]]
    # Make central BEV movie
    make_central_movie(
        pcs_all,
        tracks_all["central"],
        ego=platforms_all[ego_agent],
        extent=extent,
        vid_folder=vid_folder,
        colormethod="channel-4",
    )
    # Make central marginal movies
    print("Making centralized tracking movie marginals")
    extent = [[-70, 70], [-70, 70], [-100, 100]]
    for agent in agents:
        # AVstack movie
        make_agent_movies(
            imgs=imgs_all[agent],
            pcs=pcs_all[agent],
            dets=[],
            tracks=tracks_all["central"],
            agent=agent,
            extent=extent,
            percep_movies=False,
            track_movies=True,
            img_movies=True,
            bev_movies=True,
            vid_folder=vid_folder,
            suffix="-from-central",
        )


# collaborative perception movie
if run_collaborative_tracking:
    print("Making collaborative tracking movies")
    extent = [[-150, 20], [-80, 40], [-100, 100]]
    make_collab_movie(
        pcs_all,
        tracks_all["collab"],
        ego=platforms_all[ego_agent],
        extent=extent,
        vid_folder=vid_folder,
        colormethod="channel-4",
    )